In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
mexican_resturant_df = pd.read_csv('/content/drive/MyDrive/CIS_509/mexican.csv')

In [ ]:
mexican_resturant_df.head()

,business_id,name,category,stars,text
0,VeFfrEZ4iWaecrQg6Eq4cg,Cal Taco,Mexican,positive,The burgers are amazing! French fries are dece...
1,VeFfrEZ4iWaecrQg6Eq4cg,Cal Taco,Mexican,negative,"This was our first time trying Cal Taco, as we..."
2,VeFfrEZ4iWaecrQg6Eq4cg,Cal Taco,Mexican,positive,Best Onion Rings in Santa Barbara/Goleta!!! Ra...
3,VeFfrEZ4iWaecrQg6Eq4cg,Cal Taco,Mexican,negative,"they started serving breakfast, and its a good..."
4,VeFfrEZ4iWaecrQg6Eq4cg,Cal Taco,Mexican,positive,This place has been around for years and it's ...


In [ ]:
mexican_resturant_df.shape

(4962, 5)

In [ ]:
mexican_resturant_df.dtypes

business_id    object
name           object
category       object
stars          object
text           object
dtype: object

# Text Processing

Overall, this code snippet is preparing the text data by lowering case sensitivity, removing stopwords and punctuation, and ensuring that only meaningful, tokenized words are left for the subsequent analysis stages, thereby enhancing the quality and relevance of the text data for NLP tasks.

In [ ]:
# Define stopwords and preprocessing function
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())

    #sees that only meaningful tokens are present in token variable
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]

    # return value is joining tokens back to a string
    return " ".join(tokens)

# Apply text preprocessing to the 'text' column
mexican_resturant_df['text'] = mexican_resturant_df['text'].apply(preprocess_text)

# Training

In [ ]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 16.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 16.9 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 7.3 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039294 sha256=10d1eaaedef0f10e22044b37608ef5ba756d153db3d5bf0029a4814a86ddf026
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
  Created wheel for umap-

In [ ]:
%%time
from bertopic import BERTopic
model = BERTopic(verbose=True,embedding_model='paraphrase-MiniLM-L3-v2', min_topic_size= 7, calculate_probabilities=True)
topics, probs = model.fit_transform(mexican_resturant_df.text)

2024-02-26 09:38:30,061 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.04k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/156 [00:00<?, ?it/s]

2024-02-26 09:40:07,240 - BERTopic - Embedding - Completed ✓
2024-02-26 09:40:07,242 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-02-26 09:40:49,590 - BERTopic - Dimensionality - Completed ✓
2024-02-26 09:40:49,593 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-02-26 09:40:50,019 - BERTopic - Cluster - Completed ✓
2024-02-26 09:40:50,034 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-02-26 09:40:50,423 - BERTopic - Representation - Completed ✓


CPU times: user 2min 39s, sys: 4.53 s, total: 2min 43s
Wall time: 2min 50s


In [ ]:
#provides dataframe for the resturant category as Mexican with the info of the topic
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,4943,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...
1,1,11,1_que_la_el_es,"[que, la, el, es, muy, en, de, bueno, pero, bien]",[este restaurante mexicano es bastante bueno e...
2,2,8,2_ist_und_das_mit,"[ist, und, das, mit, lecker, nicht, ein, sehr,...",[ein überfüllter laden der leider nicht mit de...


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Topic'].plot(kind='hist', bins=20, title='Topic')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Count'].plot(kind='hist', bins=20, title='Count')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('Name').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_11['Topic'].plot(kind='line', figsize=(8, 4), title='Topic')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_12['Count'].plot(kind='line', figsize=(8, 4), title='Count')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_13['Name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_13, x='index', y='Name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['Name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='Topic', y='Name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['Name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='Count', y='Name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
model.get_topic(0)

[('food', 0.05620274996150411),
 ('place', 0.04536909516703518),
 ('good', 0.043432273287682975),
 ('great', 0.03844997630601035),
 ('tacos', 0.02829058801193655),
 ('like', 0.024006338897590285),
 ('get', 0.023774960259892924),
 ('one', 0.02310568415150837),
 ('go', 0.022428398601314237),
 ('really', 0.02173804927569978)]

In [ ]:
model.get_document_info(mexican_resturant_df.text)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,burgers amazing french fries decent get onion ...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,1.000000,False
1,first time trying cal taco moved town 's 3 min...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,0.916143,False
2,best onion rings santa barbara/goleta rare com...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,1.000000,False
3,started serving breakfast good deal two eggs t...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,1.000000,False
4,place around years 's still best local burger/...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,1.000000,False
...,...,...,...,...,...,...,...,...
4957,ohhhhh foodies delight favorite julia child tr...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,0.955440,False
4958,overhyped ferrari parked lot outside 's popula...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,0.980877,False
4959,food sanctuary santa barbara 'm convinced chor...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,0.974877,False
4960,n't know food breads yummy try bakeries around...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,0.955519,False


In [ ]:
topics[:5]

[0, 0, 0, 0, 0]

In [ ]:
probs[:5]

array([[1.00000000e+000, 2.16802912e-309, 2.19678631e-309],
       [9.16143032e-001, 4.19745781e-002, 4.18823896e-002],
       [1.00000000e+000, 2.16657934e-309, 2.19539981e-309],
       [1.00000000e+000, 2.01169252e-309, 2.01891566e-309],
       [1.00000000e+000, 2.12519995e-309, 2.14783526e-309]])

#topic reduction after training

In [ ]:
model.reduce_topics(mexican_resturant_df.text, nr_topics=20)

2024-02-26 10:14:39,228 - BERTopic - Topic reduction - Reducing number of topics
2024-02-26 10:14:39,244 - BERTopic - Topic reduction - Reduced number of topics from 3 to 3


In [ ]:
topics = model.topics_

In [ ]:
topics[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
model.probabilities_[:10]

array([[1.00000000e+000, 2.16802912e-309, 2.19678631e-309],
       [9.16143032e-001, 4.19745781e-002, 4.18823896e-002],
       [1.00000000e+000, 2.16657934e-309, 2.19539981e-309],
       [1.00000000e+000, 2.01169252e-309, 2.01891566e-309],
       [1.00000000e+000, 2.12519995e-309, 2.14783526e-309],
       [1.00000000e+000, 1.98710562e-309, 1.99520571e-309],
       [1.00000000e+000, 2.08763166e-309, 2.10530055e-309],
       [9.39073148e-001, 3.04016171e-002, 3.05252346e-002],
       [9.79815149e-001, 1.00814912e-002, 1.01033600e-002],
       [1.00000000e+000, 2.16296440e-309, 2.19194011e-309]])

In [ ]:
model.get_document_info(mexican_resturant_df.text)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,burgers amazing french fries decent get onion ...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,1.000000,False
1,first time trying cal taco moved town 's 3 min...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,0.916143,False
2,best onion rings santa barbara/goleta rare com...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,1.000000,False
3,started serving breakfast good deal two eggs t...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,1.000000,False
4,place around years 's still best local burger/...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,1.000000,False
...,...,...,...,...,...,...,...,...
4957,ohhhhh foodies delight favorite julia child tr...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,0.955440,False
4958,overhyped ferrari parked lot outside 's popula...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,0.980877,False
4959,food sanctuary santa barbara 'm convinced chor...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,0.974877,False
4960,n't know food breads yummy try bakeries around...,0,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...,food - place - good - great - tacos - like - g...,0.955519,False


# Topic Extraction and Representation

In [ ]:
freq = model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq.head()

Number of topics: 3


,Topic,Count,Name,Representation,Representative_Docs
0,0,4943,0_food_place_good_great,"[food, place, good, great, tacos, like, get, o...",[love padaro beach grill came first time june ...
1,1,11,1_que_la_el_es,"[que, la, el, es, muy, en, de, bueno, pero, bien]",[este restaurante mexicano es bastante bueno e...
2,2,8,2_ist_und_das_mit,"[ist, und, das, mit, lecker, nicht, ein, sehr,...",[ein überfüllter laden der leider nicht mit de...


In [ ]:
a_topic = freq.iloc[1]["Topic"] # Select the 1st topic
model.get_topic(a_topic) # Show the words and their c-TF-IDF scores

[('que', 0.21565216476579907),
 ('la', 0.1911502211864118),
 ('el', 0.17395529435663581),
 ('es', 0.1735028416476753),
 ('muy', 0.167593040309144),
 ('en', 0.15089546024468434),
 ('de', 0.1263742533434748),
 ('bueno', 0.09717789364072531),
 ('pero', 0.08918176212452258),
 ('bien', 0.08918176212452258)]

In [ ]:
from sklearn.cluster import KMeans

# Assuming embeddings are available
num_clusters = 5  # Example number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(embeddings)

# Attach cluster labels to the original data
sampled_data['cluster'] = clusters

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Generate a word cloud for each cluster
for i in range(num_clusters):
    cluster_texts = sampled_data[sampled_data['cluster'] == i]['cleaned_text'].str.cat(sep=' ')
    wordcloud = WordCloud(width=800, height=400, background_color ='white').generate(cluster_texts)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(f'Word Cloud for Cluster {i}')
    plt.show()